In [3]:
import os
import PIL
import re
from db import *
from PIL import ImageFont
from PIL import Image
from PIL import ImageDraw
from google.cloud import translate_v2 as translate


os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "C:\\Users\\joman\\anaconda3\\envs\\ocr2\\google_key.json"

KeyError: 'AIMBOX'

In [17]:
from google.cloud import translate_v2 as translate
translate_client = translate.Client()
result = translate_client.translate('hi', target_language='th')
print(result["translatedText"])

สวัสดี


## Calling from DB to get raw text data

In [18]:
def get_raw_text():          ## add in parameters to limit the amt of raw text
    qstr = f"""
    select raw_text from aimbox 
    where outlet_id=94
    limit 80
    """
    return aimbox.query(text=qstr)

data = get_raw_text()
data

,raw_text
0,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
1,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
2,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
3,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
4,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
...,...
95,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
96,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
97,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
98,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...


In [19]:
font_dir='C:\\Users\\joman\\Desktop\\easyocr\\font\\'
fonts = []

for root, directories, files in os.walk(font_dir):
        for filename in files:
            filepath = root+'\\'+filename
            name = filename.replace(' ','_')
            new=root+'\\'+name
            fonts.append(name)
            os.rename(filepath, new)
fonts

['Aksaramatee_Bold.ttf',
 'Aksaramatee_Italic.ttf',
 'Aksaramatee_Regular.ttf',
 'Athiti_Medium.ttf',
 'ChulaNarak_Regular.ttf',
 'Garuda.ttf',
 'TH_Sarabun_New_Regular.ttf']

In [6]:
# from google.cloud import translate_v2 as translate
# translate_client = translate.Client()
# result = translate_client.translate('hi', target_language='th')
# print(result["translatedText"])

In [42]:
def translate_to_th_img(data, text_dir, img_dir, font_dir, labels, img_name, fonts, status): #name inclusive of txt
    temp = data.split('\n') #change depending on cols of data.
    f= open(text_dir+img_name,"w+", encoding="utf-8")
    for x in range(len(temp)):
        # print(i)
        # print()
        if temp[x] != '':
            try:
                translation = translate_client.translate(str(temp[x]), target_language='th')
                f.write('\n')
                f.write(translation["translatedText"])
            except IndexError:
                continue
            except AttributeError:
                continue
    f.close()
    for i in fonts:
        font = ImageFont.truetype(font_dir+str(i), 25)
        img_loc=img_dir+img_name+'_'+i.split('.')[0]
        # print('img_loc: '+img_loc)
        print(str(img_loc), translation["translatedText"])
        # if status == 'train':
        #     labels_train=labels_train.append({'filename':str(img_loc)+'.png', 'raw_text': data.replace('\n', ' ')}, ignore_index=True)  #removing \n
        # if status == 'test':
        labels=labels.append({'filename':str(img_loc)+'.png', 'words': data.replace('\n', '')}, ignore_index=True)  #removing \n

        with open(text_dir+img_name, encoding="utf8") as f:
            lines = f.readlines()
            size = os.path.getsize(text_dir+img_name)
        empt=''
        string=empt.join(lines)
        img = Image.new(mode='RGB', size=(1000, int(size*0.7)), color='white')
        d = ImageDraw.Draw(img)
        d.text((100, 20), string, fill=(0, 0, 0), font=font)
        img.save(img_loc+".png", "PNG")
    return labels

In [39]:
font_short = fonts[2:5]
font_short

['Aksaramatee_Regular.ttf', 'Athiti_Medium.ttf', 'ChulaNarak_Regular.ttf']

In [43]:
directory='all_text_data'
# test_dir = 'C:/Users/joman/Desktop/easyocr/final_to_upload/data_test/'
train_dir = 'all_data/data_train/'
valid_dir = 'all_data/data_valid/'
font_dir='all_fonts'
labels_train = pd.DataFrame(columns=['filename', 'words'])
labels_test = pd.DataFrame(columns=['filename', 'words'])
labels_valid = pd.DataFrame(columns=['filename', 'words'])

In [ ]:
for i in range(len(data)):
    # if i%5==0:
    # test = round(len(data)*0.15)
    valid = round(len(data)*0.15)
    train = len(data) - valid
    # if i <=test:
    #     labels_test=translate_to_th_img(data['raw_text'][i], directory, test_dir, font_dir, labels_test, 'thai'+str(i), font_short, 'test')
    if i<=valid:
        labels_valid=translate_to_th_img(data['raw_text'][i], directory, valid_dir, font_dir, labels_valid, 'thai'+str(i), font_short, 'test')
    else:
        labels_train=translate_to_th_img(data['raw_text'][i], directory, train_dir, font_dir, labels_train, 'thai'+str(i), font_short, 'train')

# labels_test.to_csv('labels.csv', index=False)
labels_train.to_csv('all_data/data_train/labels.csv', index=False)
labels_valid.to_csv('all_data/data_valid/labels.csv', index=False)
print('done')

C:/Users/joman/Desktop/easyocr/final_to_upload/data_test/thai0_Aksaramatee_Regular ,1615358026
C:/Users/joman/Desktop/easyocr/final_to_upload/data_test/thai0_Athiti_Medium ,1615358026
C:/Users/joman/Desktop/easyocr/final_to_upload/data_test/thai0_ChulaNarak_Regular ,1615358026
C:/Users/joman/Desktop/easyocr/final_to_upload/data_test/thai1_Aksaramatee_Regular ,1594456795
C:/Users/joman/Desktop/easyocr/final_to_upload/data_test/thai1_Athiti_Medium ,1594456795
C:/Users/joman/Desktop/easyocr/final_to_upload/data_test/thai1_ChulaNarak_Regular ,1594456795
C:/Users/joman/Desktop/easyocr/final_to_upload/data_test/thai2_Aksaramatee_Regular ,1613376832
C:/Users/joman/Desktop/easyocr/final_to_upload/data_test/thai2_Athiti_Medium ,1613376832
C:/Users/joman/Desktop/easyocr/final_to_upload/data_test/thai2_ChulaNarak_Regular ,1613376832
C:/Users/joman/Desktop/easyocr/final_to_upload/data_test/thai3_Aksaramatee_Regular ,1615357991
C:/Users/joman/Desktop/easyocr/final_to_upload/data_test/thai3_Athiti_M

In [10]:
with open('C:/Users/joman/Desktop/easyocr/final_to_upload/gt_train.txt', 'w+', encoding="utf8") as f:
    labels_train = pd.read_csv(train_dir+'labels.csv')
    for i in range(len(labels_train)):
        f.write(labels_train['filename'][i])
        f.write('\t')
        f.write(labels_train['raw_text'][i])
        f.write('\n')
    f.close()

with open('C:/Users/joman/Desktop/easyocr/final_to_upload/gt_test.txt', 'w+', encoding="utf8") as f:
    labels_test = pd.read_csv(test_dir+'labels.csv')
    for i in range(len(labels_test)):
        f.write(labels_test['filename'][i])
        f.write('\t')
        f.write(labels_test['raw_text'][i])
        f.write('\n')
    f.close()
    
with open('C:/Users/joman/Desktop/easyocr/final_to_upload/gt_valid.txt', 'w+', encoding="utf8") as f:
    labels_valid = pd.read_csv(valid_dir+'labels.csv')
    for i in range(len(labels_valid)):
        f.write(labels_valid['filename'][i])
        f.write('\t')
        f.write(labels_valid['raw_text'][i])
        f.write('\n')
    f.close()

In [11]:
labels_test['filename'][1]

'C:/Users/joman/Desktop/easyocr/img/test/img/thai0_Aksaramatee_Italic.png'

## Single line text image

In [11]:
data

,raw_text
0,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
1,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
2,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
3,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
4,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
5,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
6,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
7,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
8,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
9,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...


In [12]:
curr = data[0:3]
curr

,raw_text
0,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
1,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...
2,FRANKIE'S NY WINGS INC.\nOwned & Optd by: Fran...


Every line is an image by itself. --> split acc to '\n' and run translate_to_th

In [9]:
# curr['raw_text'][0].split('\n')

In [10]:
round(63*0.15)

9

In [13]:
curr['raw_text'][0].split('\n')[0]

translatessss = translate.Client()
result = translatessss.translate(curr['raw_text'][0].split('\n')[0], target_language='th')
print(result["translatedText"])

แฟรงกี้ส์ นิวยอร์ก วิงส์ อิงค์


In [40]:
def translate_to_th_img_single(data, text_dir, img_dir, font_dir, labels, img_name, fonts, status): #name inclusive of txt
    temp = data.split('\n') #change depending on cols of data.
    f= open(text_dir+img_name,"w+", encoding="utf-8")
    for x in range(len(temp)):
        # print(i)
        # print()
        if temp[x] != '':
            string=re.sub(r'[\W_]+', ' ', temp[x])#.replace(' ', '') ## removing non-alphanumeric and spaces in the sentence
            try:
                translation = translate_client.translate(string, target_language='th')
                f.write('\n')
                f.write(translation["translatedText"])
            except IndexError:
                continue
            except AttributeError:
                continue
    f.close()
    for i in fonts:
        font = ImageFont.truetype(font_dir+str(i), 25)
        img_loc=img_dir+img_name+'_'+i.split('.')[0]
        # print('img_loc: '+img_loc)
        print(str(img_loc))
        # if status == 'train':
        #     labels_train=labels_train.append({'filename':str(img_loc)+'.png', 'raw_text': data.replace('\n', ' ')}, ignore_index=True)  #removing \n
        # if status == 'test':
        print(str(img_loc), translation["translatedText"])
        labels=labels.append({'filename':str(img_loc).strip()+'.png', 'words': translation["translatedText"]}, ignore_index=True)  #removing \n

        with open(text_dir+img_name, encoding="utf8") as f:
            lines = f.readlines()
            size = os.path.getsize(text_dir+img_name)
        empt=''
        string=empt.join(lines)
        img = Image.new(mode='RGB', size=(1000, 100), color='white')
        d = ImageDraw.Draw(img)
        d.text((50, 20), string, fill=(0, 0, 0), font=font)
        img.save(img_loc+".png", "PNG")
    # labels.rename(columns={'raw_text':'words'}, inplace=True)
    return labels

In [54]:
# transl = translate_client.translate('Fort Bonifacio Taguig City', target_language='th')
# trans = transl["translatedText"]
# trans

In [25]:
# fonttt = ImageFont.truetype('C:\\Users\\joman\\Desktop\\easyocr\\font\\Aksaramatee_Bold.ttf', 30)
# img = Image.new(mode='RGB', size=(1000, 70), color='white')
# d = ImageDraw.Draw(img)
# string = 'Bonifacio Taguig City'
# d.text((100, 20), string.replace(' ', ''), fill=(0, 0, 0), font=fonttt)
# img.save('C:\\Users\\joman\\Desktop\\easyocr\\single_line\\ho'+".png", "PNG")

In [41]:
single_text_dir = 'C:/Users/joman/Desktop/easyocr/single_line/text/'
single_test_img = 'C:/Users/joman/Desktop/easyocr/single_line/data_test/test/'
single_valid_img = 'C:/Users/joman/Desktop/easyocr/single_line/data_valid/test/'
single_ttrain_img = 'C:/Users/joman/Desktop/easyocr/single_line/data_train/training/'
# single_ttest_img = 'C:/Users/joman/Desktop/easyocr/single_line/data_train/testing/'
# single_tvalid_img = 'C:/Users/joman/Desktop/easyocr/single_line/data_train/validation/'
font_dir='C:/Users/joman/Desktop/easyocr/font/'

single_labels_valid = pd.DataFrame(columns=['filename', 'words'])
single_labels_test = pd.DataFrame(columns=['filename', 'words'])
single_labels_ttrain = pd.DataFrame(columns=['filename', 'words'])
# single_labels_ttest = pd.DataFrame(columns=['filename', 'words'])
# single_labels_tvalid = pd.DataFrame(columns=['filename', 'words'])

In [15]:
fonts_temp = fonts[0:1]
fonts_temp

['Aksaramatee_Bold.ttf']

In [43]:
for i in range(len(curr)): 
    temp = curr['raw_text'][i].split('\n')
    for x in range(len(temp)):
        test = round(len(temp)*0.15)
        valid = round(len(temp)*0.15)
        train = len(temp) - test - valid
        if temp[x] != '':
            if x<test:
                single_labels_test = translate_to_th_img_single(temp[x], single_text_dir, single_test_img, font_dir, single_labels_test, str(i)+'_thai'+str(x), fonts_temp, 'test')
            elif x>=test and x<test+valid:
                single_labels_valid = translate_to_th_img_single(temp[x], single_text_dir, single_valid_img, font_dir, single_labels_valid, str(i)+'_thai'+str(x), fonts_temp, 'valid')
            # elif x>=test+valid and x<round(train*0.15)+test+valid: #train_test
            #     single_labels_ttest = translate_to_th_img_single(temp[x], single_text_dir, single_ttest_img, font_dir, single_labels_ttest, str(i)+'_thai'+str(x), fonts_temp, 'valid')
            # elif x>=round(train*0.15)+test+valid and x<2*(round(train*0.15))+test+valid: #train_valid
            #     single_labels_tvalid = translate_to_th_img_single(temp[x], single_text_dir, single_tvalid_img, font_dir, single_labels_tvalid, str(i)+'_thai'+str(x), fonts_temp, 'valid')
            else: #train_train
                single_labels_ttrain = translate_to_th_img_single(temp[x], single_text_dir, single_ttrain_img, font_dir, single_labels_ttrain, str(i)+'_thai'+str(x), fonts_temp, 'train')

                
                
single_labels_test.to_csv('C:/Users/joman/Desktop/easyocr/data_thaisingle_test/data_lmdb_release/evaluation/singlelineeval\\labels.csv', index=False)
single_labels_valid.to_csv('C:/Users/joman/Desktop/easyocr/data_thaisingle_test/data_lmdb_release/validation\\labels.csv', index=False)
single_labels_ttrain.to_csv('C:/Users/joman/Desktop/easyocr/data_thaisingle_test/data_lmdb_release/training/singlelinetemp\\single_train\\labels.csv', index=False)
# single_labels_ttest.to_csv('C:/Users/joman/Desktop/easyocr/data_thaisingle_test/data_lmdb_release/training/singlelinetemp\\single_test\\labels.csv', index=False)
# single_labels_tvalid.to_csv('C:/Users/joman/Desktop/easyocr/data_thaisingle_test/data_lmdb_release/training/singlelinetemp\\single_valid\\labels.csv', index=False)
print('done')            

C:/Users/joman/Desktop/easyocr/single_line/data_test/test/0_thai0_Aksaramatee_Bold
C:/Users/joman/Desktop/easyocr/single_line/data_test/test/0_thai0_Aksaramatee_Bold แฟรงกี้ เอส นิวยอร์ก วิงส์ อิงค์
C:/Users/joman/Desktop/easyocr/single_line/data_test/test/0_thai1_Aksaramatee_Bold
C:/Users/joman/Desktop/easyocr/single_line/data_test/test/0_thai1_Aksaramatee_Bold เป็นเจ้าของ Optd โดย Frankie s NY Wings Inc
C:/Users/joman/Desktop/easyocr/single_line/data_test/test/0_thai2_Aksaramatee_Bold
C:/Users/joman/Desktop/easyocr/single_line/data_test/test/0_thai2_Aksaramatee_Bold FS 8 ระดับ 4 SM Aura
C:/Users/joman/Desktop/easyocr/single_line/data_test/test/0_thai3_Aksaramatee_Bold
C:/Users/joman/Desktop/easyocr/single_line/data_test/test/0_thai3_Aksaramatee_Bold โบนิฟาซิโอ โกลบอล ซิตี้
C:/Users/joman/Desktop/easyocr/single_line/data_test/test/0_thai4_Aksaramatee_Bold
C:/Users/joman/Desktop/easyocr/single_line/data_test/test/0_thai4_Aksaramatee_Bold ป้อม Bonifacio Taguig City
C:/Users/joman/Deskto

In [31]:
# single_labels_train = single_labels_ttest.append(single_labels_tvalid).append(single_labels_ttrain).rename(columns={'raw_text':'words'})
# single_labels_train
# single_labels_train.to_csv('C:/Users/joman/Desktop/easyocr/data_thaisingle_test/data_lmdb_release/training/singlelinetemp\\single_test\\labels.csv', index=False)

In [44]:
with open('C:\\Users\\joman\\Desktop\\easyocr\\single_line\\data_test\\gt_test.txt', 'w+', encoding="utf8") as f:
    for i in range(len(single_labels_test)):
        f.write(single_labels_test['filename'][i])
        f.write('\t')
        f.write(single_labels_test['words'][i])
        f.write('\n')
    f.close()
    
with open('C:\\Users\\joman\\Desktop\\easyocr\\single_line\\data_valid\\gt_valid.txt', 'w+', encoding="utf8") as f:
    for i in range(len(single_labels_valid)):
        f.write(single_labels_valid['filename'][i])
        f.write('\t')
        f.write(single_labels_valid['words'][i])
        f.write('\n')
    f.close()
    

with open('C:\\Users\\joman\\Desktop\\easyocr\\single_line\\data_train\\gt_ttrain.txt', 'w+', encoding="utf8") as f:
    for i in range(len(single_labels_ttrain)):
        f.write(single_labels_ttrain['filename'][i])
        f.write('\t')
        f.write(single_labels_ttrain['words'][i])
        f.write('\n')
    f.close()
    
with open('C:\\Users\\joman\\Desktop\\easyocr\\single_line\\data_train\\gt_tvalid.txt', 'w+', encoding="utf8") as f:
    for i in range(len(single_labels_tvalid)):
        f.write(single_labels_tvalid['filename'][i])
        f.write('\t')
        f.write(single_labels_tvalid['words'][i])
        f.write('\n')
    f.close()
    
with open('C:\\Users\\joman\\Desktop\\easyocr\\single_line\\data_train\\gt_ttest.txt', 'w+', encoding="utf8") as f:
    for i in range(len(single_labels_ttest)):
        f.write(single_labels_ttest['filename'][i])
        f.write('\t')
        f.write(single_labels_ttest['words'][i])
        f.write('\n')
    f.close()

In [35]:
char_loaded = open('C:\\Users\\joman\\Desktop\\easyocr\\characters.txt', "r", encoding="utf-8")
lines = char_loaded.readlines()
lines

['0123456789abcdefghijklmnopqrstuvwxyzกขฃคฅฆงจฉชซฌญฎฏฐฑฒณดตถทธนบปผฝพฟภมยรลวศษสหฬอฮอะอิอึอุเอะแอะโอะเอาะเออะเอียะเอือะอัวะอำไอใอเอาอาอีอือูเอแอโอออเออเอียเอืออัว']

## CLI for creation of lmdb dataset

In [36]:
!cd C:/Users/joman/Documents/GitHub/deep-text-recognition-benchmark

In [27]:
!python C:/Users/joman/Documents/GitHub/deep-text-recognition-benchmark/create_lmdb_dataset.py \
--inputPath C:/Users/joman/Desktop/easyocr/single_line/data_test/ \
--gtFile C:/Users/joman/Desktop/easyocr/single_line/data_test/gt_test.txt \
--outputPath C:/Users/joman/Desktop/easyocr/data_thaisingle_test/data_lmdb_release/evaluation/singlelineeval/

Created dataset with 28 samples


In [28]:
!python C:/Users/joman/Documents/GitHub/deep-text-recognition-benchmark/create_lmdb_dataset.py \
--inputPath C:/Users/joman/Desktop/easyocr/single_line/data_valid/ \
--gtFile C:/Users/joman/Desktop/easyocr/single_line/data_valid/gt_valid.txt \
--outputPath C:/Users/joman/Desktop/easyocr/data_thaisingle_test/data_lmdb_release/validation/

Created dataset with 30 samples


In [29]:
!python C:/Users/joman/Documents/GitHub/deep-text-recognition-benchmark/create_lmdb_dataset.py \
--inputPath C:/Users/joman/Desktop/easyocr/single_line/data_train/ \
--gtFile C:/Users/joman/Desktop/easyocr/single_line/data_train/gt_ttest.txt \
--outputPath C:/Users/joman/Desktop/easyocr/data_thaisingle_test/data_lmdb_release/training/singlelinetemp/single_test/

Created dataset with 22 samples


In [30]:
!python C:/Users/joman/Documents/GitHub/deep-text-recognition-benchmark/create_lmdb_dataset.py \
--inputPath C:/Users/joman/Desktop/easyocr/single_line/data_train/ \
--gtFile C:/Users/joman/Desktop/easyocr/single_line/data_train/gt_ttrain.txt \
--outputPath C:/Users/joman/Desktop/easyocr/data_thaisingle_test/data_lmdb_release/training/singlelinetemp/single_train/

Created dataset with 77 samples


In [31]:
!python C:/Users/joman/Documents/GitHub/deep-text-recognition-benchmark/create_lmdb_dataset.py \
--inputPath C:/Users/joman/Desktop/easyocr/single_line/data_train/ \
--gtFile C:/Users/joman/Desktop/easyocr/single_line/data_train/gt_tvalid.txt \
--outputPath C:/Users/joman/Desktop/easyocr/data_thaisingle_test/data_lmdb_release/training/singlelinetemp/single_valid/ 

Created dataset with 22 samples


In [42]:
!python C:/Users/joman/Documents/GitHub/deep-text-recognition-benchmark/train.py \
--train_data C:/Users/joman/Desktop/easyocr/data_thaisingle_test/data_lmdb_release/training \
--valid_data C:/Users/joman/Desktop/easyocr/data_thaisingle_test/data_lmdb_release/validation \
--select_data singlelinetemp \
--batch_ratio 1 \
--Transformation None \
--FeatureExtraction VGG \
--SequenceModeling BiLSTM \
--Prediction CTC \
--data_filtering_off

device count 0

Traceback (most recent call last):
  File "C:\Users\joman\Documents\GitHub\deep-text-recognition-benchmark\train.py", line 348, in <module>
    train(opt)
  File "C:\Users\joman\Documents\GitHub\deep-text-recognition-benchmark\train.py", line 234, in train
    print(lines)
  File "C:\Users\joman\anaconda3\envs\ocr2\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode characters in position 0-7: character maps to <undefined>



training
--------------------------------------------------------------------------------
dataset_root: C:/Users/joman/Desktop/easyocr/data_thaisingle_test/data_lmdb_release/training
opt.select_data: ['singlelinetemp']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    C:/Users/joman/Desktop/easyocr/data_thaisingle_test/data_lmdb_release/training	 dataset: singlelinetemp
sub-directory:	/singlelinetemp\single_test	 num samples: 22
sub-directory:	/singlelinetemp\single_train	 num samples: 77
sub-directory:	/singlelinetemp\single_valid	 num samples: 22
num total samples of singlelinetemp: 121 x 1.0 (total_data_usage_ratio) = 121
num samples of singlelinetemp per batch: 192 x 1.0 (batch_ratio) = 192
--------------------------------------------------------------------------------
Total_batch_size: 192 = 192
--------------------------------------------------------------------------------
dataset_root:    C:/Users/joman/De

## Getting gt_file for images

In [38]:
directory = 'C:\\Users\\joman\\Desktop\\easyocr\\data_english_test\\training\\eval'

In [39]:
#getting directory of every image
import os

def get_gt(directory):
    file_paths = []  # List which will store all of the full filepaths.
    with open('C:\\Users\\joman\\Desktop\\easyocr\\data_english_test\\training\\gt_train_eval.txt', 'w+', encoding="utf8") as f:
    # Walk the tree.
        for root, directories, files in os.walk(directory):
            for filename in files:
                # Join the two strings in order to form the full filepath.
                filepath = root+'\\'+filename
                name = filename.split('_')[0]
                if name.isalpha()==True:
                    f.write(filepath)
                    f.write('\t')
                    f.write(name)
                    f.write('\n')
                else:
                    os.remove(filepath)

In [40]:
get_gt(directory)